In [1]:
"""!git clone https://github.com/FabriDeCastelli/HLT-News-Category.git
! pip install keras-tuner

import os
os.chdir("/content/HLT-News-Category/src/test")"""

'!git clone https://github.com/FabriDeCastelli/HLT-News-Category.git\n! pip install keras-tuner\n\nimport os\nos.chdir("/content/HLT-News-Category/src/test")'

In [2]:
import warnings
import sys
import os


warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
from src.main.models.bidirLSMT import BidirectionalLSTM
from src.main.pipeline.functions import clean_text, remove_contractions, stop_words_removal, tokenize
from src.main.utilities.utils import get_dataset, load_pretrained_embeddings, split_train_val_test, embedding_matrix_statistics
import keras as K
from config.config import LOGS_PATH


import warnings
warnings.filterwarnings("ignore")

In [5]:
bidirLSTM = BidirectionalLSTM(pretrained_embeddings="glove")

bidirLSTM.pipeline = [remove_contractions, clean_text, stop_words_removal, tokenize]

inputs, targets = get_dataset(one_hot=True)
inputs = bidirLSTM.run_pipeline(inputs, save=True)

# stratified train test split
x_train, x_val, x_test, y_train, y_val, y_test = split_train_val_test(inputs, targets, validation_size=0.2, test_size=0.1)

Creating embedding matrix...


In [3]:
import random

pretrained_embeddings = load_pretrained_embeddings("glove")
find_ratio, unmatched_words = embedding_matrix_statistics(pretrained_embeddings)

print("Find ratio:", find_ratio)
print("Unmatched tokens count:", len(unmatched_words))
print("Unmatched words examples:\n" + str(random.sample(unmatched_words, 10)))
print(str(random.sample(unmatched_words, 10)))

Find ratio: 0.823297298907055
Unmatched tokens count: 11867
Unmatched words examples:
['cravenly', 'averys', 'ekhlas', 'dpp4', 'caff', 'burningman', 'zaller', 'uberstar', 'tweaky', 'xiangshawan']
['ylvis', 'chewiness', 'resortsare', 'celsion', 'kohans', 'aughts', 'justiceforjane', 'manspreaders', 'opensecrets', 'dowds']


In [4]:
callbacks = [
    K.callbacks.EarlyStopping(monitor='f1_macro', patience=2),
    K.callbacks.TensorBoard(LOGS_PATH.format(repr(bidirLSTM)))
]
best_hps = bidirLSTM.grid_search(x_train, y_train, x_val, y_val, callbacks=callbacks, n_iter=50)
best_hps.values

Reloading Tuner from /Users/fabriziodecastelli/Desktop/Magistrale/HLT/HLT-News-Category/results/BidirectionalLSTM/BidirectionalLSTM/tuner0.json

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
100               |60                |lstm_units_1
30                |40                |lstm_units_2
0.4               |0.4               |dropout1
0.1               |0.1               |dropout2
40                |40                |dense1
20                |30                |dense2
0.001             |0.0001            |learning_rate
5                 |5                 |epochs
512               |1024              |batch_size

Epoch 1/5
 22/148 ━━━━━━━━━━━━━━━━━━━━ 1:53 903ms/step - accuracy: 0.3812 - f1_macro: 0.0299 - loss: 1.4867 - precision_macro: 0.0680 - recall_macro: 0.0222

KeyboardInterrupt: 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "{LOGS_PATH.format(repr(bidirLSTM))}" --host localhost

In [ ]:
bidirLSTM.save_results(x_test, y_test)

In [ ]:
bidirLSTM.save_model()

In [4]:
BidirectionalLSTM.get_top_experiments("BidirectionalLSTM", 5)

[({'lstm_units_1': 100, 'lstm_units_2': 30, 'dropout1': 0.4, 'dropout2': 0.1, 'dense1': 40, 'dense2': 20, 'learning_rate': 0.001, 'epochs': 5, 'batch_size': 512}, None), ({'lstm_units_1': 60, 'lstm_units_2': 40, 'dropout1': 0.4, 'dropout2': 0.1, 'dense1': 40, 'dense2': 30, 'learning_rate': 0.0001, 'epochs': 5, 'batch_size': 1024}, 0.0), ({'lstm_units_1': 120, 'lstm_units_2': 60, 'dropout1': 0.4, 'dropout2': 0.2, 'dense1': 20, 'dense2': 30, 'learning_rate': 0.001, 'epochs': 5, 'batch_size': 1024}, None), ({'lstm_units_1': 80, 'lstm_units_2': 40, 'dropout1': 0.4, 'dropout2': 0.3, 'dense1': 60, 'dense2': 20, 'learning_rate': 0.0001, 'epochs': 10, 'batch_size': 1024}, 0.5), ({'lstm_units_1': 140, 'lstm_units_2': 20, 'dropout1': 0.3, 'dropout2': 0.2, 'dense1': 60, 'dense2': 20, 'learning_rate': 0.01, 'epochs': 10, 'batch_size': 1024}, 0.5), ({'lstm_units_1': 80, 'lstm_units_2': 40, 'dropout1': 0.3, 'dropout2': 0.2, 'dense1': 20, 'dense2': 30, 'learning_rate': 0.0005, 'epochs': 5, 'batch_siz

TypeError: '<' not supported between instances of 'NoneType' and 'float'